## Perform power analysis given observed linear model effect sizes

In [1]:
library(pwr)
suppressPackageStartupMessages(library(dplyr))

In [2]:
plate <- "localhost220513100001_KK22-05-198_FactinAdjusted"

lm_results_file <- file.path("results", paste0(plate, "_linear_model_cp_features.tsv"))
output_file <- file.path("results", paste0(plate, "_power_analysis_cp_features_lm.tsv"))

In [3]:
# Load linear model results
lm_results_df <- readr::read_tsv(
    lm_results_file,
    col_types = readr::cols(.default="d", feature="c")
)

print(dim(lm_results_df))
head(lm_results_df)

[1] 505   4


feature,r2_score,cell_count_coef,treatment_dose_coef
<chr>,<dbl>,<dbl>,<dbl>
Cytoplasm_AreaShape_Compactness,2.647033e-02,4.030892e-04,-0.050195942
Cytoplasm_AreaShape_FormFactor,7.008457e-02,-8.281876e-04,0.074347347
Cytoplasm_AreaShape_MajorAxisLength,5.938975e-02,-1.345084e-03,0.025608622
Cytoplasm_AreaShape_MinorAxisLength,6.112660e-02,-1.350586e-03,0.027654612
Cytoplasm_AreaShape_Orientation,8.552248e-05,6.251026e-06,0.003491483
Cytoplasm_AreaShape_Zernike_0_0,2.856302e-02,-2.018002e-04,0.058909125


In [4]:
# Load feature data (for calculating n)
file_suffix = "_sc_norm_fs_cellprofiler_ic.csv.gz"
data_dir = file.path("..", "..", "..", "3.process_cfret_features", "data")
cp_file <- file.path(data_dir, paste0(plate, file_suffix))

cp_df <- readr::read_csv(
    cp_file,
    col_types = readr::cols(
        .default="d",
        Metadata_WellRow="c",
        Metadata_WellCol="c",
        Metadata_heart_number="c",
        Metadata_treatment="c",
        Metadata_dose="c",
        Metadata_Plate="c",
        Metadata_Well="c"
    )
)

print(dim(cp_df))
head(cp_df, 3)

New names:
• `` -> `...1`


[1] 17995   645


...1,Metadata_WellRow,Metadata_WellCol,Metadata_number_of_singlecells,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,⋯,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumEntropy_ER_3_03_256,Nuclei_Texture_SumEntropy_Hoechst_3_00_256,Nuclei_Texture_SumEntropy_Mitochondria_3_03_256,Nuclei_Texture_SumEntropy_PM_3_01_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_01_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_PM_3_01_256
<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,A,9,357,9,drug_x,5uM,1,localhost220513100001_KK22-05-198_FactinAdjusted,A09,⋯,-1.2788029,-0.8897645,0.5844583,-0.7568114,1.6462852,1.6277442,-0.1302769,-0.32954013,0.16874427,0.22393503
1,A,9,357,9,drug_x,5uM,1,localhost220513100001_KK22-05-198_FactinAdjusted,A09,⋯,0.1400462,0.1777536,-1.0356146,-0.4881881,-0.9279062,-0.3785298,-0.4520701,-0.30924955,-0.17971123,-0.19329881
2,A,9,357,9,drug_x,5uM,1,localhost220513100001_KK22-05-198_FactinAdjusted,A09,⋯,-0.7494158,-1.6935892,0.3866221,0.9995438,1.1926342,1.1170768,-0.1127250,0.04806898,-0.02098493,0.01024755


## Perform power analysis

In [5]:
# Define constants for power analysis
n_conditions <- length(table(cp_df$Metadata_dose))  # The number of doses
n_samples <- dim(cp_df)[1]
alpha_standard <- 0.05

u <- n_conditions - 1
v <- n_samples - u - 1
sig_level <- alpha_standard / dim(lm_results_df)[1]  # Multiple test adjusted
power <- 0.8

print(c(u, v))
print(sig_level)

[1]     9 17985
[1] 9.90099e-05


In [6]:
# Given all R2 values perform power analysis
all_power_results <- list()
for (cp_feature in lm_results_df$feature) {
    # Subset to the given feature lm results
    lm_result_subset_df <- lm_results_df %>%
        dplyr::filter(feature == !!cp_feature)
    
    # Pull out the estimated R2 value
    r2_val <- lm_result_subset_df %>% dplyr::pull(r2_score)
    
    # The power estimate is undefined for r2_val = 1, skip if so
    if (r2_val == 1) {
        all_power_results[[cp_feature]] <- c(cp_feature, u, v, sig_level, NULL, NULL)
        next
    }
    
    # Transform R2 score to F2 effect size
    f2_val <- r2_val / (1 - r2_val)
    
    # Calculate power, note that v contains an estimate of sample size
    power_result <- pwr.f2.test(u = u, v = NULL, f2 = f2_val, sig.level = sig_level, power = power)
    
    # Calculate required sample size from the v formula
    estimated_sample_size <- power_result$v + u + 1
    
    # Save results for future visualization
    all_power_results[[cp_feature]] <- c(cp_feature, u, v, sig_level, power, estimated_sample_size)
    
}

In [7]:
# Compile and output results
power_results_df <- do.call(rbind, all_power_results) %>% dplyr::as_tibble()

colnames(power_results_df) <- c("feature", "u", "v", "sig_level", "power", "estimated_sample_size")

# Output to file
power_results_df %>%
    readr::write_tsv(output_file)

print(dim(power_results_df))
head(power_results_df)

Warning message:
“The `x` argument of `as_tibble.matrix()` must have unique column names if
`.name_repair` is omitted as of tibble 2.0.0.
ℹ Using compatibility `.name_repair`.”


[1] 505   6


feature,u,v,sig_level,power,estimated_sample_size
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Cytoplasm_AreaShape_Compactness,9,17985,9.9009900990099e-05,0.8,1326.94521098789
Cytoplasm_AreaShape_FormFactor,9,17985,9.9009900990099e-05,0.8,489.531770286284
Cytoplasm_AreaShape_MajorAxisLength,9,17985,9.9009900990099e-05,0.8,581.053058719846
Cytoplasm_AreaShape_MinorAxisLength,9,17985,9.9009900990099e-05,0.8,564.011687359225
Cytoplasm_AreaShape_Orientation,9,17985,9.9009900990099e-05,0.8,416473.260741323
Cytoplasm_AreaShape_Zernike_0_0,9,17985,9.9009900990099e-05,0.8,1228.35388406464
